<a href="https://colab.research.google.com/github/VayuSarangam/cmbs-cashflow/blob/main/notebooks/01_loan_level_cashflow_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ALWAYS FIRST STEP: Start a fresh Colab session, clone with GitHub repository (You should see data files saved in GitHub populated as a result of clone)

In [ ]:
%cd /content
!rm -rf cmbs-cashflow
!git clone https://github.com/VayuSarangam/cmbs-cashflow.git
%cd /content/cmbs-cashflow
!pip -q install -r requirements.txt
!ls -lah data
!ls -lah src


/content
Cloning into 'cmbs-cashflow'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 43 (delta 13), reused 12 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 41.18 KiB | 1.29 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/cmbs-cashflow
total 40K
drwxr-xr-x 2 root root 4.0K Feb 15 19:43 .
drwxr-xr-x 7 root root 4.0K Feb 15 19:43 ..
-rw-r--r-- 1 root root  230 Feb 15 19:43 cmbs_deal_terms.csv
-rw-r--r-- 1 root root 7.2K Feb 15 19:43 cmbs_loan_tape_new_issue.csv
-rw-r--r-- 1 root root    1 Feb 15 19:43 .gitkeep
-rw-r--r-- 1 root root  141 Feb 15 19:43 OUTPUT_collateral_cashflows_TEMPLATE.csv
-rw-r--r-- 1 root root  198 Feb 15 19:43 OUTPUT_loan_level_projection_TEMPLATE.csv
-rw-r--r-- 1 root root  783 Feb 15 19:43 README.txt
-rw-r--r-- 1 root root  334 Feb 15 19:43 scenario_shocks.csv
total 20K
drwxr-xr-x 2 root root 4.0K Feb 15 19:43 .
drwxr-xr-x 7 root r

Check to see if anything not saved

In [5]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Create Notebook 01 inside the GitHub repo, open file from GitHub, run code to clone if needed to pupulate data folder or refresh.

Confirm notebook saved:

In [6]:
!mkdir -p notebooks
!ls -lah notebooks


total 32K
drwxr-xr-x 2 root root 4.0K Feb 15 20:15 .
drwxr-xr-x 7 root root 4.0K Feb 15 20:15 ..
-rw-r--r-- 1 root root  18K Feb 15 20:15 01_loan_level_cashflow_projection.ipynb
-rw-r--r-- 1 root root    1 Feb 15 20:15 .gitkeep


Load inputs

In [7]:
import pandas as pd
from src.loan_projection import run_projection

loan_tape = pd.read_csv("data/cmbs_loan_tape_new_issue.csv")
deal_terms = pd.read_csv("data/cmbs_deal_terms.csv")
scenarios = pd.read_csv("data/scenario_shocks.csv")

loan_tape.shape, deal_terms.shape, scenarios.shape


((35, 22), (1, 9), (4, 9))

Run projection

That line runs your entire **loan-level projection engine** and returns two tables.

## What `run_projection(...)` is doing

Inputs:

* `loan_tape`: one row per loan at cutoff (balance, rate, IO flag, amort term, fee bps, etc.)
* `deal_terms`: deal-level settings (cutoff date, projection horizon, recovery lag)
* `scenarios`: scenario assumptions (Annual CPR, Annual CDR, Severity)

`run_projection` then does:

### 1) Build the monthly timeline

* Reads `CutoffDate` and `ProjectionHorizonMonths`
* Creates a list of **month-end dates** starting next month-end, for `horizon` months

### 2) Loop scenarios

For each scenario (Base, Stress, etc.):

* Convert annual assumptions to monthly:

  * **SMM (Single Monthly Mortality)** from CPR:
    `SMM = 1 - (1 - CPR)^(1/12)`
  * **MDR (Monthly Default Rate)** from CDR:
    `MDR = 1 - (1 - CDR)^(1/12)`

### 3) Loop months, then loans

For each month and each loan, it projects a single period:

**a) Interest**

* `gross_interest = beginning_balance * note_rate / 12`

**b) Servicing fee**

* `servicing_fee = beginning_balance * (ServicingFeeBps / 10,000) / 12`

**c) Net interest**

* `net_interest = gross_interest - servicing_fee`

**d) Scheduled principal**

* If IOFlag = Y → scheduled principal = 0
* Else it computes a level monthly payment using the amort term, and sets:
  `scheduled_principal = payment - gross_interest`

**e) Default**

* `default_principal = MDR * beginning_balance`
* `realized_loss = severity * default_principal`
* `recovery_amount = (1 - severity) * default_principal`
* That recovery is not paid immediately. It is queued and paid **after RecoveryLagMonths**.

**f) Prepayment**

* Applies after default on the surviving balance:

  * `prepay_base = beginning_balance - default_principal`
  * `prepayment_principal = SMM * prepay_base`

**g) Ending balance**
Principal that reduces balance is:

* scheduled principal + prepay principal + default principal
  Then:
* `ending_balance = beginning_balance - (sched + prepay + default)`

### 4) Store outputs in two tables

#### `loan_df` (loan-level)

One row per:

* Scenario × LoanID × Month

Contains columns like:

* BeginningBalance, GrossInterest, ServicingFee, NetInterest
* ScheduledPrincipal, PrepaymentPrincipal, DefaultPrincipal
* RecoveryAmount (paid this month from past defaults)
* RealizedLoss
* EndingBalance

This matches your loan-level template.

#### `pool_df` (deal-level collateral aggregation)

One row per:

* Scenario × Month

It sums across loans for that month:

* InterestCollected = sum(NetInterest)
* PrincipalCollected = sum(ScheduledPrincipal + PrepaymentPrincipal + DefaultPrincipal)
* Recoveries = sum(RecoveryAmount paid this month)
* RealizedLosses = sum(RealizedLoss)
* FeesPaid = sum(ServicingFee)
* EndingCollateralBalance = sum(EndingBalance)

This matches your collateral template.

## What `loan_df.head(), pool_df.head()` does

It just prints the first 5 rows of each dataframe so you can sanity-check the numbers.

---

## Two important modeling caveats (so you understand what you’re building)

1. This is **expected cashflows**, not Monte Carlo. Defaults and prepays are applied as fractional rates every month.
2. Loss is recorded when default happens, but **recoveries are delayed** by the lag queue.

If that makes sense, you’re ready for Step 5 (template enforcement + write outputs).


In [8]:
loan_df, pool_df = run_projection(loan_tape, deal_terms, scenarios)

loan_df.head(), pool_df.head()


(            DealID Scenario    LoanID PeriodEndDate  BeginningBalance  \
 0  CMBS_NEW_AEOYEE     Base  NI_L0001    2026-02-28      5.505704e+07   
 1  CMBS_NEW_AEOYEE     Base  NI_L0002    2026-02-28      2.201915e+07   
 2  CMBS_NEW_AEOYEE     Base  NI_L0003    2026-02-28      5.451226e+07   
 3  CMBS_NEW_AEOYEE     Base  NI_L0004    2026-02-28      6.769117e+07   
 4  CMBS_NEW_AEOYEE     Base  NI_L0005    2026-02-28      1.600318e+08   
 
    GrossInterest  ServicingFee  NetInterest  ScheduledPrincipal  \
 0      381315.90      11470.22    369845.68            34672.69   
 1      104976.31       4587.32    100388.99            33161.97   
 2      236446.93      11356.72    225090.21           129511.75   
 3      352388.96      14102.33    338286.64            94023.46   
 4     1032738.36      33339.95    999398.41           175397.31   
 
    PrepaymentPrincipal  DefaultPrincipal  RecoveryAmount  RealizedLoss  \
 0            380594.73          92613.79             0.0      37045.

confirm your actual template columns  

In [9]:
pd.read_csv("data/OUTPUT_loan_level_projection_TEMPLATE.csv").columns.tolist(), \
pd.read_csv("data/OUTPUT_collateral_cashflows_TEMPLATE.csv").columns.tolist()


(['DealID',
  'Scenario',
  'LoanID',
  'PeriodEndDate',
  'BeginningBalance',
  'GrossInterest',
  'ServicingFee',
  'NetInterest',
  'ScheduledPrincipal',
  'PrepaymentPrincipal',
  'DefaultPrincipal',
  'RecoveryAmount',
  'RealizedLoss',
  'EndingBalance',
  'Status'],
 ['DealID',
  'Scenario',
  'PeriodEndDate',
  'InterestCollected',
  'PrincipalCollected',
  'Recoveries',
  'RealizedLosses',
  'FeesPaid',
  'NetCollections',
  'EndingCollateralBalance'])

## Match templates + write outputs

Here’s exactly what this Step is doing, line by line, and why it matters.

---

## 1) Load the templates

```python
tmpl_loan = pd.read_csv("data/OUTPUT_loan_level_projection_TEMPLATE.csv")
tmpl_pool = pd.read_csv("data/OUTPUT_collateral_cashflows_TEMPLATE.csv")
```

* These “template” CSVs usually contain **only headers** (no rows).
* When pandas reads them, you get two empty DataFrames whose main value is:

  * `tmpl_loan.columns` = the **official required column names + order** for `loan_df`
  * `tmpl_pool.columns` = the **official required column names + order** for `pool_df`

So you’re not loading data. You’re loading the **schema**.

---

## 2) Force your outputs to match the schema (columns + order)

```python
loan_df = loan_df[tmpl_loan.columns]
pool_df = pool_df[tmpl_pool.columns]
```

This does two things at once:

### A) Reorders columns

Even if your `loan_df` has all the right columns, they might be in a different order.
This forces them into the exact template order.

### B) Validates required columns exist

If your `loan_df` is missing *any* column in the template, pandas throws a `KeyError`.

That’s good. It stops you from exporting a “wrong” file that will break later notebooks (waterfall/pricing).

### C) Drops extra columns (silently)

If your `loan_df` has extra debug columns not in the template, they get removed here.

So after this line:

* your output CSV will contain **only what you want**
* in the **exact expected order**

---

## 3) Ensure the outputs folder exists

```python
import os
os.makedirs("outputs", exist_ok=True)
```

* Creates a folder called `outputs/` if it doesn’t exist.
* `exist_ok=True` means: if it already exists, don’t error.

This prevents failure when you try to write CSVs into a missing directory.

---

## 4) Write output files to disk

```python
loan_df.to_csv("outputs/loan_level_projection.csv", index=False)
pool_df.to_csv("outputs/collateral_cashflows.csv", index=False)
```

* Writes your two DataFrames into CSVs inside `outputs/`.
* `index=False` prevents pandas from writing the row index as an extra unwanted column (common mistake).

Result:

* `outputs/loan_level_projection.csv` → loan-level monthly projection
* `outputs/collateral_cashflows.csv` → pooled collateral cashflows (sum across loans)

---

## 5) Print row/column counts

```python
loan_df.shape, pool_df.shape
```

This returns:

* `(num_rows, num_cols)` for each DataFrame

It’s a quick sanity check:

* `loan_df` rows should be roughly: `#loans × #months × #scenarios` (minus loans that hit 0 balance early)
* `pool_df` rows should be: `#months × #scenarios`

If `pool_df` doesn’t match that pattern, something’s off.

---

### Why Step 5 matters

Without Step 5, you can easily end up with:

* misspelled columns (later joins break)
* wrong column order (downstream code expecting specific layout breaks)
* extra columns (confusing, inconsistent artifacts)

Step 5 is your **schema lock** + **export step**.


In [10]:
import pandas as pd

tmpl_loan = pd.read_csv("data/OUTPUT_loan_level_projection_TEMPLATE.csv")
tmpl_pool = pd.read_csv("data/OUTPUT_collateral_cashflows_TEMPLATE.csv")

loan_df = loan_df[tmpl_loan.columns]
pool_df = pool_df[tmpl_pool.columns]

import os
os.makedirs("outputs", exist_ok=True)

loan_df.to_csv("outputs/loan_level_projection.csv", index=False)
pool_df.to_csv("outputs/collateral_cashflows.csv", index=False)

loan_df.shape, pool_df.shape


((16800, 15), (480, 10))